In [7]:
import requests
import pandas as pd

from datetime import datetime

from openai import OpenAI
from bs4 import BeautifulSoup
import html5lib

In [2]:
client = OpenAI(
    api_key="" # PUT YOUR API KEY HERE
)

In [11]:
# Cole Palmer
player_name = 'Cole Palmer'
url = 'https://fbref.com/en/players/dc7f8a28/Cole-Palmer'
attrs = 'scout_summary_AM'




df = pd.read_html(
    url, 
    attrs={'id': attrs} # Change the id & class to correspond to the table
)[0]

In [13]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [22]:
# Get the player's age, team, league, and position
position = soup.select_one('p:-soup-contains("Position")').text.split('▪')[0].split(':')[1].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()
# get the player's age
age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()

'Chelsea'

In [27]:
prompt = f"""

I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: {player_name}
Position: {position}
Age: {age}
Team: {team}

{df.to_markdown()}


Return the scouting report in the following markdown format:

# Scouting Report for {player_name}

## Strengths
< a list of 1 to 3 strengths >

## Weaknesses
< a list of 1 to 3 weaknesses >

## Summary
< a brief summary of the player's overall performance and if he would be beneficial to the team or atleast a good investment for a future sell >
"""

In [31]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a professional football (soccer) scout."},
        {"role": "user", "content": prompt},
    ],
)

In [32]:
print(response.choices[0].message.content)

# Scouting Report for Cole Palmer

## Strengths
- **Scoring and Playmaking Abilities**: Cole Palmer exhibits a strong scoring ability, shown by his high percentile in Non-Penalty Goals (89th) and Non-Penalty xG (96th). His potential for providing assists is also impressive, ranking in the 94th percentile with a similar percentile in expected assisted goals.
- **Shooting Volume and Precision**: His ability to take shots is among the best, with a total shots per 90 in the 91st percentile. This, combined with his solid pass completion percentage and number of progressive passes, allows him to consistently position himself as a scoring threat.
- **Creative Output**: Palmer is highly creative, as evidenced by his shot-creating actions per 90 in the 90th percentile and xG+xA in the 96th percentile. His proficiency in creating and taking part in offensive plays is a significant asset.

## Weaknesses
- **Physical and Defensive Contributions**: His physical presence in duels and aerial contests